<a href="https://colab.research.google.com/github/aliasgharchakera/GPU-Spring24-Assignment01/blob/main/AY06993_Assgn1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-jditb33z
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-jditb33z
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5cd225851b7638f3f6d55a19328295f16c014079
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.0.3-py3-none-any.whl size=7432 sha256=ad99cf1e6175f37f13398802e9e82d42885745e385f9d1c2ce678f3d359f3a95
  Stored in directory: /tmp/pip-ephem-wheel-cache-xodwdj24/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter
Source files will be saved in "/tmp/tmp4lqcsf_c".


In [3]:
!nvidia-smi

Fri Jan 26 18:00:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
%%cuda
#include <stdio.h>
#include <curand_kernel.h>

inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
  if (err != cudaSuccess) {
    fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
  }
  return err;
}


__global__ void InitData(int* data)
{
    curandState_t state;
    curand_init(clock64(), 0, 0, &state);
    data[threadIdx.x] = curand(&state) % 100;
}

int main()
{
  const int N = 1000000;
  double time1, timedif;

  int* h_data = (int*)malloc(N * sizeof(int));
  int* d_data;

  time1 = (double) clock();            /* get initial time */
  for (int i = 0; i < N; i++)
    {
        h_data[i] = rand() % 100;
    }

  /* call clock a second time */
  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);

  //allocate memory on device
  cudaMalloc((void **)&d_data, N*sizeof(int));

  //call kernel
  time1 = (double) clock();
  InitData<<<1000,N/1000>>>(d_data);

  checkCudaErr(cudaDeviceSynchronize(), "Syncronization");
  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);
  checkCudaErr(cudaGetLastError(), "GPU Error");

  time1 = (double) clock();
  //copy device data to host memory
  checkCudaErr(cudaMemcpy(h_data, d_data, N*sizeof(int), cudaMemcpyDeviceToHost), "cudaMemcpy D->H");
  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);
  /*
  printf("Data (on device): \n");
  for(int i=0; i<N; ++i)
     printf(" data[%d] -> %d\n", i, h_data[i]);
  */
  //release GPU memory
  cudaFree(d_data);
  free(h_data);


  return 0;
}

The elapsed time is 0.025627 seconds
The elapsed time is 0.268397 seconds
The elapsed time is 0.001062 seconds



In [11]:
%%cuda
#include <stdio.h>
#include <time.h>

__global__ void sum(int* a, int* b, int* c, const int N) {
	int i = threadIdx.x + blockDim.x * blockIdx.x;
	if (i<N)
		c[i] = a[i] + b[i];
	//else
	//	printf("i > N -> %3d in block: %d\n", i, blockIdx.x);
}

void sum_host(int* a, int* b, int* c, const int N) {
  clock_t clk;

	clk = clock();
	for(int i=0; i<N; ++i)
		c[i] = a[i] + b[i];

	clk = clock() - clk;
	double dt = (((double)clk) / CLOCKS_PER_SEC) * 1000;
	printf("Total time on CPU: %f msecs\n", dt);

}


int main() {
	int* h_a = 0;
	int* h_b = 0;
	int* h_c = 0;

	int* d_a = 0;
	int* d_b = 0;
	int* d_c = 0;

	const int N = 1000;
  double time1, timedif;
	int numThreadsPerBlock =  128;
	int numBlocksPerGrid = ceilf(N/numThreadsPerBlock + 1);
	printf("Num threads per block: %3d\n", numThreadsPerBlock);
	printf("Num blocks per grid: %3d\n", numBlocksPerGrid);

	size_t size = N * sizeof(int);

	//allocate host memory
	h_a = (int*)malloc(size);
	h_b = (int*)malloc(size);
	h_c = (int*)malloc(size);

	//initialize a, b and c
	//for(int i=0;i<N;++i) {
	//	h_a[i] = i+1;
	//	h_b[i] = h_a[i]*2;
	//	h_c[i] = 0;
	//}

	//allocate device memory
	cudaMalloc((void**)&d_a, size);
	cudaMalloc((void**)&d_b, size);
	cudaMalloc((void**)&d_c, size);

	//copy host data to device memory
	//cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
	//cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);

	//calculate on host
  time1 = (double) clock();
	sum_host(h_a, h_b, h_c, N);
  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);

	printf("Sum (host): %d\n", h_c);


	//calculate on device
  time1 = (double) clock();
	sum<<<numBlocksPerGrid, numThreadsPerBlock>>>(d_a, d_b, d_c, N);
	cudaDeviceSynchronize();

  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);


	//copy result from device to host
  time1 = (double) clock();
	cudaMemcpy(h_c, d_c, size, cudaMemcpyDeviceToHost);
  timedif =  ((double) clock() - time1) / CLOCKS_PER_SEC;
  printf("The elapsed time is %lf seconds\n", timedif);

  printf("Sum (device): %d\n", h_c);

	//delete data allocated on device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	//delete host memory
	free(h_a);
	free(h_b);
	free(h_c);

	return 0;
}

Num threads per block: 128
Num blocks per grid:   8
Total time on CPU: 0.007000 msecs
The elapsed time is 0.000014 seconds
Sum (host): 1303033072
The elapsed time is 0.000246 seconds
The elapsed time is 0.000025 seconds
Sum (device): 1303033072

